In [1]:
import findspark
findspark.init()
import pyspark

In [2]:
spark = pyspark.sql.SparkSession.builder.appName('lreg').getOrCreate()

22/06/14 10:44:14 WARN Utils: Your hostname, mehrdad-Standard resolves to a loopback address: 127.0.1.1; using 128.179.176.230 instead (on interface wlp4s0)
22/06/14 10:44:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/14 10:44:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/14 10:44:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/06/14 10:44:16 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/06/14 10:44:16 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/06/14 10:44:16 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/06/

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
training = spark.read.format('libsvm').load('../data/sample_linear_regression_data.txt')

22/06/14 10:44:19 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [5]:
# create Linear Regression model
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [6]:
lr_model = lr.fit(training)

22/06/14 10:44:26 WARN Instrumentation: [c9087fb1] regParam is zero, which might cause numerical instability and overfitting.


In [7]:
lr_model.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [8]:
lr_model.intercept

0.14228558260358096

In [9]:
training_summary = lr_model.summary

In [10]:
training_summary.r2

0.027839179518600154

In [11]:
training_summary.rootMeanSquaredError

10.16309157133015

### Train / Test Split

In [12]:
all_data = spark.read.format('libsvm').load('../data/sample_linear_regression_data.txt')

22/06/14 10:44:28 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [13]:
train, test = all_data.randomSplit([0.7, 0.3])

In [14]:
train.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                347|
|   mean|0.30698707019394195|
| stddev| 10.357316565600062|
|    min|-28.571478869743427|
|    max| 26.903524792043335|
+-------+-------------------+



In [15]:
test.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                154|
|   mean|0.14400510754800722|
| stddev| 10.261204910896971|
|    min|-19.782762789614537|
|    max|  27.78383192005107|
+-------+-------------------+



### Train Linear Regression & Evaluate on Test Set

In [16]:
better_model = lr.fit(train)

22/06/14 10:44:31 WARN Instrumentation: [24a21c9c] regParam is zero, which might cause numerical instability and overfitting.


In [17]:
test_results = better_model.evaluate(test)

In [18]:
test_results.residuals.show()

/opt/spark/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+-------------------+
|          residuals|
+-------------------+
|-19.745253702444398|
| -19.33839639371503|
|-18.701397402104035|
|-17.601337967496523|
|-17.123370708027938|
|  -15.7227953129177|
|-15.826768025097602|
|-19.815047073087392|
|-16.756687663892862|
|-17.600981643910615|
|-16.381414768936285|
|-16.404426763097423|
|-15.852442626343775|
| -13.05708242509704|
|-15.479569171122597|
|-14.658669012300276|
| -13.74598037524442|
|-14.720725120140681|
|-12.272787625604387|
|-11.957771568589168|
+-------------------+
only showing top 20 rows



In [19]:
test_results.rootMeanSquaredError

10.399571191666356

In [20]:
test_results.r2

-0.033864049596737056

### Deploying a Model

In [21]:
unlabeled_data = test.select('features')

In [22]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [23]:
predictions = better_model.transform(unlabeled_data)

In [24]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|-0.03750908717014109|
|(10,[0,1,2,3,4,5,...|-0.32892222165669105|
|(10,[0,1,2,3,4,5,...| -0.7009386281105182|
|(10,[0,1,2,3,4,5,...|  0.2746172348205739|
|(10,[0,1,2,3,4,5,...| 0.05797108215192351|
|(10,[0,1,2,3,4,5,...| -1.3036969512918477|
|(10,[0,1,2,3,4,5,...| -0.8923288085074873|
|(10,[0,1,2,3,4,5,...|  3.1228400517762847|
|(10,[0,1,2,3,4,5,...|  0.6053383126157494|
|(10,[0,1,2,3,4,5,...|  1.5153226028891256|
|(10,[0,1,2,3,4,5,...| 0.42990220314171057|
|(10,[0,1,2,3,4,5,...|  0.6809111500488543|
|(10,[0,1,2,3,4,5,...|  0.4150578329125576|
|(10,[0,1,2,3,4,5,...| -2.2538981643192497|
|(10,[0,1,2,3,4,5,...|  0.4230861965801639|
|(10,[0,1,2,3,4,5,...|-0.16348389745091288|
|(10,[0,1,2,3,4,5,...| -1.0167778776867071|
|(10,[0,1,2,3,4,5,...|  0.8536372249819131|
|(10,[0,1,2,3,4,5,...| -1.1478071502863694|
|(10,[0,1,2,3,4,5,...| -1.195564